# Шаг 1
### Подключение необходимых библиотек

In [1]:
import math
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from scipy.special import comb, gamma
import scipy.stats as stats


# Шаг 2
### Функции для генерации случайных данных раличных распределений

In [2]:
def dataset_generator(name, seed, size):
    rnd_gen = np.random.default_rng(seed)
    # Геометническое распределение
    if (name == 'geometric'):
        p = rnd_gen.random()
        return name, rnd_gen.geometric(p, size=size)
    # Биномиальное распределение
    elif (name == 'binomial'):
        n = rnd_gen.integers(1, 1000)
        p = rnd_gen.random()
        return name, rnd_gen.binomial(n, p, size=size)
    # Распределение Пуассона
    elif (name == 'poisson'):
        expected_value = rnd_gen.integers(100)
        return name, rnd_gen.poisson(expected_value, size=size)
    # Равномерное распределение
    elif (name == 'uniform'):
        a = rnd_gen.integers(10)
        b = a + rnd_gen.integers(1, 10)
        return name, rnd_gen.uniform(a, b, size=size)
    # Нормальное распределение
    elif (name == 'normal'):
        expected_value = rnd_gen.integers(-10, 10)
        # Среднеквадратичное отклонение
        standart_deviation = rnd_gen.integers(1, 100)
        return name, rnd_gen.normal(expected_value, standart_deviation, size=size)
    # Экспоненциальное распределение
    elif (name == 'exponential'):
        expected_value = rnd_gen.integers(-10, 10)
        return name, rnd_gen.exponential(expected_value, size=size)

    
    
name, data = dataset_generator('uniform', seed=8, size=20)
print(name)
print(data)

uniform
[9.96183053 7.95613252 9.36564681 9.60968954 8.17325442 8.31364562
 8.11824671 7.32086079 8.43689636 7.72405644 7.77143575 7.55419467
 7.58159365 9.44148301 8.26895256 7.76776152 8.77270856 8.81281689
 8.94057408 9.73406914]


# Шаг 3
### Функции плотности распределения известных распределений

In [3]:
def normal_distribution(mean: float, variance: float, deviation: float):    
    return lambda x: 1 / (deviation * np.sqrt(2 * np.pi)) * np.exp(-(x - mean) ** 2 / (2 * variance)), 2

def exponential_distribution(mean: float):
    intensity = 1 / mean
    if intensity <= 0:
        return None
    else:
        return lambda x: intensity * math.exp(-intensity * x) if x >= 0 else 0, 1

def uniform_distribution(mean: float, variance: float):
    b = (3 * variance) ** 0.5 + mean
    a = 2 * mean - b   
    return lambda x: 1 / (b - a) if a <= x <= b else 0, 2 

def geometric_distribution(mean: float):
    p = 1 / mean
    if p > 1 or p <= 0:
        return None
    else:
        q = 1 - p
        return lambda x: q ** x * p if x >= 0 else 0, 3

def binomial_distribution(mean: float, n: int):
    p = mean / n
    if p > 1 or p < 0 or n < 0:
        return None
    else:
        return lambda x: comb(n, x) * (p ** x) * ((1 - p) ** (n - x)) if x >= 0 else 0, 2

def poisson_distribution(mean: float):
    return lambda x: math.exp(-mean) * (mean ** x) / gamma(x + 1) if x >= 0 else 0, 1

# Шаг 4
### Функция для разделения наблюдаемых значений на разряды
### Вовращает массив, в котором каждый элемент содержит данные разряда и среднеей

In [4]:
def split_into_intervals(lst, bins_count):
    if len(lst) == 0:
        return []

    sorted_lst = sorted(lst)
    
    res = []

    interval_len = (sorted_lst[-1]-sorted_lst[0]) / bins_count
    counter = sorted_lst[0]
    
    for i in sorted_lst: 
        if i >= counter:
            while counter <= i:      
                counter += interval_len

            res.append([[], 0, 0])
            
        res[-1][0].append(i)
        res[-1][1] += i
    
    for i in res:
        i[1] = ((i[0][0] + i[0][-1]) / 2) if len(i[0]) else 0
        i[2] = len(i[0]) / len(lst)
    
    return (res, interval_len)

print(split_into_intervals([0, 1, 1, 1, 7, 2, 8, 3, 3, 9, 4, 10, 5, 11, 6, 12], 6))

([[[0, 1, 1, 1], 0.5, 0.25], [[2, 3, 3], 2.5, 0.1875], [[4, 5], 4.5, 0.125], [[6, 7], 6.5, 0.125], [[8, 9], 8.5, 0.125], [[10, 11], 10.5, 0.125], [[12], 12.0, 0.0625]], 2.0)


# Шаг 5
### Функция для получения мат ожидания, дисперсии и среднего отклонения

In [5]:
def get_characteristics(intervals):
    expected_val = sum(x * p for _, x, p in intervals)
    variance = sum((x - expected_val)**2 * p for _, x, p in intervals)
    deviation = math.sqrt(variance)

    return expected_val, variance, deviation